# Imports

In [2]:
import os
from functools import partial
import json

import geopandas as gpd
import matplotlib.pyplot as plt; plt.style.use("ggplot")
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import seaborn as sns
import xml.etree.ElementTree as ET
import csv

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)

from gerrychain.metrics import efficiency_gap, mean_median, partisan_gini
from gerrychain.proposals import recom
from gerrychain.updaters import cut_edges
from gerrychain.tree import recursive_tree_part
from io import BytesIO

# Downloading state data

In [302]:
graph_path = "./pennsylvania.json"
graph = Graph.from_json(graph_path)
with open (graph_path, "r") as myfile:
    data=myfile.readlines()
graph_dict = json.loads(data[0])

In [244]:
va_graph_path = "./virginia.json"
va_graph = Graph.from_json(va_graph_path)
with open (va_graph_path, "r") as myfile:
    data=myfile.readlines()
va_graph_dict = json.loads(data[0])

In [307]:
newdir = "./PA_Outputs/"
os.makedirs(os.path.dirname(newdir + "init.txt"), exist_ok=True)
with open(newdir + "init.txt", "w") as f:
    f.write("Created Folder")

# Chain stuff

In [305]:
pop_count = 0

for i in graph.nodes:
    pop_count += graph.nodes[i]["TOT_POP"]
    
print(pop_count)

12684929


In [258]:
va_pop_count = 0

for i in va_graph.nodes:
    va_pop_count += va_graph.nodes[i]["TOTPOP"]
    
print(pop_count)

12684929


In [306]:
%%time
num_dist = 18

# Exercise: Compute exact population from your data.
pop = pop_count

my_updaters = {
    "population": updaters.Tally("TOT_POP", alias="population"),
    "cut_edges": cut_edges,
    "SEN16": Election("SEN16", {"democratic":"T16SEND","republican":"T16SENR"}),
}


new_plan = recursive_tree_part(graph,
                               range(num_dist),
                               pop/num_dist,
                               "TOT_POP",
                               0.01,
                               3)
initial_partition = Partition(graph,
                              new_plan,
                              my_updaters)

CPU times: user 18.6 s, sys: 486 ms, total: 19 s
Wall time: 19.2 s


In [259]:
%%time
va_num_dist = 11

# Exercise: Compute exact population from your data.
va_pop = va_pop_count

va_my_updaters = {
    "population": updaters.Tally("TOTPOP", alias="population"),
    "cut_edges": cut_edges,
    "SEN18": Election("SEN18", {"democratic":"G18DSEN","republican":"G18RSEN"}),
}


va_new_plan = recursive_tree_part(va_graph,
                               range(va_num_dist),
                               va_pop/va_num_dist,
                               "TOTPOP",
                               0.01,
                               3)
va_initial_partition = Partition(va_graph,
                              va_new_plan,
                              va_my_updaters)

CPU times: user 3.6 s, sys: 34 ms, total: 3.64 s
Wall time: 3.68 s


In [308]:
%%time
proposal = partial(recom,
                   pop_col = "TOT_POP",
                   pop_target = pop/num_dist,
                   epsilon = 0.05,
                   node_repeats = 3)

compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]), 2 * len(initial_partition["cut_edges"])
)

CPU times: user 104 ms, sys: 2.06 ms, total: 106 ms
Wall time: 114 ms


In [260]:
%%time
va_proposal = partial(recom,
                   pop_col = "TOTPOP",
                   pop_target = va_pop/va_num_dist,
                   epsilon = 0.05,
                   node_repeats = 3)

va_compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]), 2 * len(initial_partition["cut_edges"])
)

CPU times: user 17 µs, sys: 1e+03 ns, total: 18 µs
Wall time: 21 µs


In [312]:
%%time
chain = MarkovChain(
    proposal=proposal,
    constraints=[
        constraints.within_percent_of_ideal_population(initial_partition, 0.05),
        compactness_bound,
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=10000,
)

CPU times: user 59 µs, sys: 1 µs, total: 60 µs
Wall time: 66 µs


In [310]:
%%time
va_chain = MarkovChain(
    proposal=va_proposal,
    constraints=[
        constraints.within_percent_of_ideal_population(initial_partition, 0.05),
        va_compactness_bound,
    ],
    accept=accept.always_accept,
    initial_state=va_initial_partition,
    total_steps=10000,
)

CPU times: user 810 µs, sys: 937 µs, total: 1.75 ms
Wall time: 3.78 ms


In [313]:
%%time

mms = []
egs = []
pgs = []
ces = []
seats = []

for step in tqdm_notebook(chain):
    mms.append(mean_median(step["SEN16"]))
    egs.append(efficiency_gap(step["SEN16"]))
    pgs.append(partisan_gini(step["SEN16"]))
    ces.append(len(step["cut_edges"]))
    seats.append(step["SEN16"].wins("republican"))

CPU times: user 1h 24min, sys: 34.6 s, total: 1h 24min 35s
Wall time: 1h 32min 42s


In [262]:
%%time

va_mms = []
va_egs = []
va_pgs = []
va_ces = []
va_seats = []


for step in tqdm_notebook(va_chain):
    va_mms.append(mean_median(step["SEN18"]))
    va_egs.append(efficiency_gap(step["SEN18"]))
    va_pgs.append(partisan_gini(step["SEN18"]))
    va_ces.append(len(step["cut_edges"]))
    seats.append(step["SEN18"].wins("republican"))

CPU times: user 6min 13s, sys: 2.08 s, total: 6min 15s
Wall time: 6min 22s


In [314]:
with open(newdir + "PAmms" + ".csv", "w") as tf1:
    tf1.writelines(str(i) + "\n" for i in mms)

with open(newdir + "PAegs"+ ".csv", "w") as tf1:
    tf1.writelines(str(j) + "\n" for j in egs)
    
with open(newdir + "PApgs" + ".csv", "w") as tf1:
    tf1.writelines(str(i) + "\n" for i in pgs)

with open(newdir + "PAces"+ ".csv", "w") as tf1:
    tf1.writelines(str(j) + "\n" for j in ces)
    
with open(newdir + "PAseats"+ ".csv", "w") as tf1:
    tf1.writelines(str(j) + "\n" for j in seats)
    
# with open(newdir + "VAmms" + ".csv", "w") as tf1:
#     tf1.writelines(str(i) + "\n" for i in va_mms)

# with open(newdir + "VAegs"+ ".csv", "w") as tf1:
#     tf1.writelines(str(j) + "\n" for j in va_egs)
    
# with open(newdir + "VApgs" + ".csv", "w") as tf1:
#     tf1.writelines(str(i) + "\n" for i in va_pgs)

# with open(newdir + "VAces"+ ".csv", "w") as tf1:
#     tf1.writelines(str(j) + "\n" for j in va_ces)
    
# with open(newdir + "VAseats"+ ".csv", "w") as tf1:
#     tf1.writelines(str(j) + "\n" for j in va_seats)

In [6]:
def save_with_pretty_fonts(filename):
    # Fix fonts
    # See https://matplotlib.org/3.1.0/gallery/user_interfaces/svg_histogram_sgskip.html
    buf = BytesIO()
    plt.savefig(buf, transparent=True, bbox_inches='tight', format='svg')

    ET.register_namespace('', 'http://www.w3.org/2000/svg')
    tree, xmlid = ET.XMLID(buf.getvalue())

    text_ids = []
    for elem in tree.iter('{http://www.w3.org/2000/svg}g'):
        if 'id' in elem.attrib:
            if elem.attrib['id'].startswith('text_'):
                for text in elem.iter('{http://www.w3.org/2000/svg}text'):
                    if 'style' in text.attrib:
                        text.attrib['style'] = 'text-anchor:middle;font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,"Noto Sans",sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji";font-size:0.7em;color:#6c757d'
    ET.ElementTree(tree).write(filename)
    plt.close()

In [324]:
mms = "./PA_Outputs/PAmms"
egs = "./PAegs"
ces = "./PAces"

In [4]:
with open("./PA_Outputs/PAmms.csv") as f:
    mms = list(float(line.strip()) for line in f)
    
with open("./PA_Outputs/PAegs.csv") as f:
    egs = list(float(line.strip()) for line in f)
    
with open("./PA_Outputs/PAces.csv") as f:
    ces = list(float(line.strip()) for line in f)
    
with open("./VA_Outputs/VAmms.csv") as f:
    va_mms = list(float(line.strip()) for line in f)
    
with open("./VA_Outputs/VAegs.csv") as f:
    va_egs = list(float(line.strip()) for line in f)
    
with open("./VA_Outputs/VAces.csv") as f:
    va_ces = list(float(line.strip()) for line in f)

In [332]:
egs

[-0.1894612654262072,
 -0.1894612654262072,
 -0.1894612654262072,
 -0.1894612654262072,
 -0.1894612654262072,
 -0.1894612654262072,
 -0.1894612654262072,
 -0.19114884356242234,
 -0.19114884356242234,
 -0.19114884356242234,
 -0.12986033818243567,
 -0.12986033818243567,
 -0.12986033818243567,
 -0.12986033818243567,
 -0.13045079186956673,
 -0.07269412546950828,
 -0.07179548881636147,
 -0.07179548881636147,
 -0.07179548881636147,
 -0.07179548881636147,
 -0.07130281066665468,
 -0.071867697605484,
 -0.071867697605484,
 -0.06723510646218565,
 -0.0662732439314818,
 -0.0662732439314818,
 -0.0662732439314818,
 -0.1245481552295488,
 -0.1245481552295488,
 -0.1245481552295488,
 -0.1245481552295488,
 -0.1245481552295488,
 -0.1245481552295488,
 -0.12446661803226212,
 -0.12446661803226212,
 -0.1855384970448812,
 -0.1855384970448812,
 -0.1840200394937162,
 -0.12582666539293588,
 -0.12582666539293588,
 -0.12957616723002766,
 -0.12957616723002766,
 -0.12957616723002766,
 -0.12642731122972778,
 -0.1254212

# Visualization

In [344]:
# Pennsylvania Median Median scores

plt.rcParams["svg.fonttype"] = "none"
fig, ax = plt.subplots(1)
sns.distplot(mms, bins=40, color="#4b89b9", ax=ax, kde=False)
ax.set_yticklabels([])
ax.axvline(x=-0.078, color="red")
ax.yaxis.set_visible(False)
ax.grid(False)
#plt.xlabel("Mean-Median Score")
#plt.ylabel("Frequency")
save_with_pretty_fonts('pa_mm.svg')

In [10]:
# Pennsylvania Median Median scores

plt.rcParams["svg.fonttype"] = "none"
fig, ax = plt.subplots(1)
sns.distplot(mms, bins=40, color="#4b89b9", ax=ax, kde=False)
ax.set_yticklabels([])
ax.axvline(x=-0.015, color="red")
ax.axvline(x=-0.065, color="green")
ax.yaxis.set_visible(False)
ax.grid(False)
#plt.xlabel("Mean-Median Score")
#plt.ylabel("Frequency")
save_with_pretty_fonts('pa_mm_tough.svg')

In [7]:
# Pennsylvania vs. Virginia Mean Median Scores

plt.rcParams["svg.fonttype"] = "none"
fig, ax = plt.subplots(1)
sns.distplot(mms, bins=40, color="#4b89b9", ax=ax, kde=False, label="PA — SEN16")
sns.distplot(va_mms, bins=40, color="#d2553e", ax=ax, kde=False, label="VA — SEN18")
ax.set_yticklabels([])
ax.axvline(x=-0.045, color="green")
ax.yaxis.set_visible(False)
ax.grid(False)
# ax.legend(loc="upper left")
#plt.xlabel("Mean-Median Score")
#plt.ylabel("Frequency")
save_with_pretty_fonts('pa_vs_va_mm.svg')

In [340]:
# Pennsylvania Efficiency Gap Scores

plt.rcParams["svg.fonttype"] = "none"
fig, ax = plt.subplots(1)
sns.distplot(egs, bins=40, color="#4b89b9", ax=ax, kde=False)
ax.set_yticklabels([])
ax.axvline(x=-0.04, color="red")
ax.yaxis.set_visible(False)
ax.grid(False)
# plt.xlabel("Efficiency Gap Score")
# plt.ylabel("Frequency")
save_with_pretty_fonts("pa_eg.svg")

In [341]:
# Pennsylvania Cut Edges Scores

plt.rcParams["svg.fonttype"] = "none"
fig, ax = plt.subplots(1)
sns.distplot(ces, bins=40, color="#4b89b9", ax=ax, kde=False)
ax.set_yticklabels([])
ax.axvline(x=1500, color="red")
ax.yaxis.set_visible(False)
ax.grid(False)
# plt.xlabel("Cut Edges")
# plt.ylabel("Frequency")
save_with_pretty_fonts("pa_ce.svg")

In [346]:
# People's heights histogram: vs. NBA players

n = 1000000
male_heights = (70 + 4 * np.random.randn(n)) / 12
female_heights = (65 + 3.5 * np.random.randn(n)) / 12
all_heights = np.concatenate((male_heights, female_heights), axis=0)
nba_heights = (79 + 3.5 * np.random.randn(2*n)) / 12

plt.rcParams['svg.fonttype'] = 'none'
fig, ax = plt.subplots(1)
# sns.distplot(male_heights, bins=100, color='#4b89b9', ax=ax, kde=False)
# sns.distplot(female_heights, bins=100, color='#d2553e', ax=ax, kde=False)
sns.distplot(all_heights, bins=100, color='#4b89b9', ax=ax, kde=False, label="all people")
sns.distplot(nba_heights, bins=100, color="red", ax=ax, kde=False, label="NBA players")
ax.set_yticklabels([])
ax.yaxis.set_visible(False)
ax.grid(False)
plt.xlim(3, 8)
ax.axvline(x=7, color="red", label="Your Friend's Height")
# ax.legend(loc="upper left")
# plt.xlabel('Height (feet)')
save_with_pretty_fonts("people_vs_nba_heights.svg")

In [347]:
# People's heights histogram: vs. NBA players

n = 1000000
male_heights = (70 + 4 * np.random.randn(n)) / 12
female_heights = (65 + 3.5 * np.random.randn(n)) / 12
all_heights = np.concatenate((male_heights, female_heights), axis=0)
nba_heights = (79 + 3.5 * np.random.randn(2*n)) / 12

plt.rcParams['svg.fonttype'] = 'none'
fig, ax = plt.subplots(1)
# sns.distplot(male_heights, bins=100, color='#4b89b9', ax=ax, kde=False)
# sns.distplot(female_heights, bins=100, color='#d2553e', ax=ax, kde=False)
sns.distplot(all_heights, bins=100, color='#4b89b9', ax=ax, kde=False, label="all people")
# sns.distplot(nba_heights, bins=100, color="red", ax=ax, kde=False, label="NBA players")
ax.set_yticklabels([])
ax.yaxis.set_visible(False)
ax.grid(False)
plt.xlim(3, 8)
ax.axvline(x=7, color="red", label="Your Friend's Height")
# ax.legend(loc="upper left")
# plt.xlabel('Height (feet)')
save_with_pretty_fonts("heights.svg")